# Transfer Learning con Inception-Style CNN – CIFAR-100

### En este notebook construiremos un modelo convolucional basado en Inception Blocks para clasificar imágenes del dataset CIFAR-100 usando sus 20 superclases.

## 🧠 Importar librerías necesarias

In [1]:
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2025-04-19 03:37:15.052003: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 03:37:15.121234: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-19 03:37:15.121280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-19 03:37:15.123980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 03:37:15.136349: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 03:37:15.138164: I tensorflow/core/platform/cpu_feature_guard.cc:1

## 📦 Cargar y preparar los datos
Cargar imágenes y etiquetas de CIFAR-100, usando superclases (20 clases)

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='coarse')

Normalización de imágenes: valores de píxeles a rango [0,1]

In [3]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

Conversión de etiquetas a codificación one-hot

In [4]:
y_train_cat = to_categorical(y_train, 20)
y_test_cat = to_categorical(y_test, 20)

## 🔥 Aumentación de datos
Crear generador de imágenes con transformaciones aleatorias para robustecer el entrenamiento

In [5]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

Crear generadores para entrenamiento y validación

In [6]:
train_generator = datagen.flow(x_train, y_train_cat, batch_size=32, subset='training')
val_generator = datagen.flow(x_train, y_train_cat, batch_size=32, subset='validation')

## 🧱 Definición de la Arquitectura CNN con Inception Blocks

In [7]:
def inception_block(x, filters):
    f1, f3, f5 = filters

    path1 = layers.Conv2D(f1, (1, 1), padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
    path1 = layers.LeakyReLU(alpha=0.1)(path1)

    path2 = layers.Conv2D(f3, (1, 1), padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
    path2 = layers.LeakyReLU(alpha=0.1)(path2)
    path2 = layers.Conv2D(f3, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4))(path2)
    path2 = layers.LeakyReLU(alpha=0.1)(path2)

    path3 = layers.Conv2D(f5, (1, 1), padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
    path3 = layers.LeakyReLU(alpha=0.1)(path3)
    path3 = layers.Conv2D(f5, (5, 5), padding='same', kernel_regularizer=regularizers.l2(1e-4))(path3)
    path3 = layers.LeakyReLU(alpha=0.1)(path3)

    path4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    path4 = layers.Conv2D(f1, (1, 1), padding='same', kernel_regularizer=regularizers.l2(1e-4))(path4)
    path4 = layers.LeakyReLU(alpha=0.1)(path4)

    return layers.concatenate([path1, path2, path3, path4], axis=-1)

## ⚙️ Construcción del modelo CNN

In [8]:
input_layer = layers.Input(shape=(32, 32, 3))

x = layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4))(input_layer)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.25)(x)

x = inception_block(x, filters=(64, 96, 128))
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.3)(x)

x = inception_block(x, filters=(64, 96, 128))
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.3)(x)

x = layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.4)(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.LeakyReLU(alpha=0.1)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

output = layers.Dense(20, activation='softmax')(x)

model = models.Model(inputs=input_layer, outputs=output)

## ⚡ Compilación del modelo


In [9]:
optimizer = SGD(learning_rate=1e-2, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## 💾 Callbacks para entrenamiento


In [10]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
checkpoint_cb = ModelCheckpoint(filepath='checkpoints/cnn_cifar100_superclass_best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

## 🚀 Entrenamiento del modelo


In [11]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint_cb, reduce_lr],
    verbose=1
)

Epoch 1/50
1250/1250 [==============================] - ETA: 0s - loss: 3.1244 - accuracy: 0.1380
Epoch 1: val_accuracy improved from -inf to 0.15640, saving model to checkpoints/cnn_cifar100_superclass_best.h5


/usr/local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 214s 169ms/step - loss: 3.1244 - accuracy: 0.1380 - val_loss: 2.8929 - val_accuracy: 0.1564 - lr: 0.0100
Epoch 2/50
1250/1250 [==============================] - ETA: 0s - loss: 2.7596 - accuracy: 0.1968
Epoch 2: val_accuracy improved from 0.15640 to 0.16960, saving model to checkpoints/cnn_cifar100_superclass_best.h5
1250/1250 [==============================] - 247s 198ms/step - loss: 2.7596 - accuracy: 0.1968 - val_loss: 2.8656 - val_accuracy: 0.1696 - lr: 0.0100
Epoch 3/50
1250/1250 [==============================] - ETA: 0s - loss: 2.5940 - accuracy: 0.2416
Epoch 3: val_accuracy improved from 0.16960 to 0.22830, saving model to checkpoints/cnn_cifar100_superclass_best.h5
1250/1250 [==============================] - 186s 149ms/step - loss: 2.5940 - accuracy: 0.2416 - val_loss: 2.6192 - val_accuracy: 0.2283 - lr: 0.0100
Epoch 4/50
1250/1250 [==============================] - ETA: 0s - loss: 2.4918 - accuracy: 0.2712
Epoch 4: val_accuracy im